In [1]:
import sys
import os
import numpy as np
import time
from imp import reload

# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")
    
%matplotlib qt

from L3 import SystemsBuilder

new directory is: C:\Users\Luke\Desktop\Barracuda\AutomatedCE


C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L1\MicroControlServer.py


 Change System Config setting if thorlabs is needed. 


In [3]:
ce_system = SystemsBuilder.CESystem()
ce_system.load_config(config_file=r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\Eclipse.cfg")

ce_system.open_controllers()

cam = ce_system.camera
shut = ce_system.excitation_shutter
filt = ce_system.filter_wheel
xy = ce_system.xy_stage
obj = ce_system.objective

items = [cam, shut, filt, xy, obj]
for utility in items:
    utility.startup()


ce_system.utilities

daq
uniphase
{'outlet_pressure': <L2.PressureControl.ArduinoPressure object at 0x00000181AD58C848>, 'xy_stage': <L2.XYControl.PycromanagerXY object at 0x00000181AD58C788>, 'objective': <L2.ZControl.PycromanagerZ object at 0x00000181AD58C908>, 'outlet_z': <L2.ZControl.ArduinoZ object at 0x00000181AD58C988>, 'inlet_z': <L2.ZControl.ArduinoZ object at 0x00000181AD58C888>, 'inlet_rgb': <L2.LightControl.RGBArduino object at 0x00000181AD58C9C8>, 'filter_wheel': <L2.FilterWheelControl.PycromanagerFilter object at 0x00000181AD58CC88>, 'excitation_shutter': <L2.ShutterControl.PycromanagerShutter object at 0x00000181AD58CB08>, 'camera': <L2.CameraControl.PycromanagerControl object at 0x00000181AD58CB88>, 'high_voltage': <L2.HighVoltageControl.SpellmanPowerSupply object at 0x00000181AD58CBC8>, 'detector': <L2.DetectorControl.PhotomultiplierDetector object at 0x00000181AD58CE48>, 'lysis_laser': <L2.LaserControl.Uniphase object at 0x00000181AD58F6C8>}
OPENING:  mmc1 <L1.Controllers.PycromanagerCont

{'outlet_pressure': <L2.PressureControl.ArduinoPressure at 0x181ad58c848>,
 'xy_stage': <L2.XYControl.PycromanagerXY at 0x181ad58c788>,
 'objective': <L2.ZControl.PycromanagerZ at 0x181ad58c908>,
 'outlet_z': <L2.ZControl.ArduinoZ at 0x181ad58c988>,
 'inlet_z': <L2.ZControl.ArduinoZ at 0x181ad58c888>,
 'inlet_rgb': <L2.LightControl.RGBArduino at 0x181ad58c9c8>,
 'filter_wheel': <L2.FilterWheelControl.PycromanagerFilter at 0x181ad58cc88>,
 'excitation_shutter': <L2.ShutterControl.PycromanagerShutter at 0x181ad58cb08>,
 'camera': <L2.CameraControl.PycromanagerControl at 0x181ad58cb88>,
 'high_voltage': <L2.HighVoltageControl.SpellmanPowerSupply at 0x181ad58cbc8>,
 'detector': <L2.DetectorControl.PhotomultiplierDetector at 0x181ad58ce48>,
 'lysis_laser': <L2.LaserControl.Uniphase at 0x181ad58f6c8>}

In [122]:
rgb_chnl = 'R'
fluor_channel = 1
wait = 0.25
exp = 250
_old_exp=100
_old_bin = 1
_old_channel = 0
bins = 8

# Set to new values
ce_system.inlet_rgb.turn_off_channel('R')
ce_system.camera.stop()
ce_system.filter_wheel.set_channel(fluor_channel)
ce_system.excitation_shutter.set_auto_on()
#_old_exp = ce_system.camera.exposure
ce_system.camera.set_exposure(exp)
ce_system.camera.set_binning(bins)
#_old_bin = ce_system.camera.bin_size
time.sleep(wait)

img = ce_system.camera.snap()

# Return to old values
ce_system.camera.set_exposure(_old_exp)
ce_system.camera.set_binning(_old_bin)
ce_system.excitation_shutter.set_auto_off()
ce_system.filter_wheel.set_channel(_old_channel)
ce_system.inlet_rgb.turn_on_channel('R')




'R:On, G:Off, B:Off'

In [12]:
import matplotlib.pyplot as plt

In [15]:
plt.imshow(img)

In [5]:
from pathlib import Path
import numpy as np
from scipy.ndimage import label, distance_transform_edt
from skimage import io, img_as_float, filters, morphology
from skimage.feature import peak_local_max
from skimage.measure import regionprops, regionprops_table
from skimage.morphology import watershed
import pandas as pd
import glob
import skimage.measure as measure

import matplotlib.pyplot as plt

In [123]:
close_size=5
open_size=5
watershed_footprint = (5,5)
# Make sure types are the same
input_image = img_as_float(img)

# Filter Image
filtered_image = filters.median(input_image, behavior='ndimage')

# Edge Detection
edge_sobel = filters.sobel(filtered_image)

# Threshold
thresh = filters.threshold_otsu(filtered_image)
binary_otsu = filtered_image > thresh

# Binary Morphology Operations
structure_element = morphology.disk(close_size)
closed_image = morphology.binary_closing(binary_otsu, structure_element)
structure_element = morphology.disk(open_size)
opened_image = morphology.binary_opening(closed_image, structure_element)

# Watershed
distance = distance_transform_edt(opened_image)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones(watershed_footprint), labels=opened_image)
markers = label(local_maxi)[0]
labels = watershed(-distance, markers, mask=opened_image)

plt.imshow(labels)

regions = regionprops_table(labels, intensity_image=img, properties=('label', 'centroid', 'max_intensity', 'mean_intensity'))


df = pd.DataFrame(regions)

In [130]:
plt.figure()
plt.imshow(filtered_image)

In [124]:
df.head()

,label,centroid-0,centroid-1,max_intensity,mean_intensity
0,1,60,87,1150.0,889.761307


In [125]:
df = df.sort_values(by=['mean_intensity','max_intensity'], ascending=False)

if df.shape[0] > 0:
    y=df['centroid-0'][0]*bins
    x=df['centroid-1'][0]*bins


In [126]:
target_x = 695
target_y = 485

movex = (x-target_x)*0.0001063
movey = (y-target_y)*0.0001063

In [127]:
movex

0.0001063

In [128]:
# 542, 679
# 540, 446
#307, 450
ce_system.xy_stage.set_rel_x(movex)


In [129]:
ce_system.xy_stage.set_rel_y(-movey)


In [50]:
0.05/470

0.00010638297872340427

In [ ]:
#837 467 60x obje
#399 479
#399 943

In [44]:
837-399

438

In [45]:
943-479

464

In [ ]:
#845 3711

In [47]:
845-371

474

In [ ]:
#487 953

In [49]:
953-487

466

In [ ]:
470